1. Install Tensorflow

In [ ]:
!python -m pip install tensorflow==2.12.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


2. Import the necessary packages to process data, train the model as well as Deploy the model

In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pylab as plt

from tensorflow import keras
from tensorflow.keras import layers

3. Mount your Google Drive ,

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


3. Path to the datatset from My Drive

In [ ]:
dataset_path = "/content/drive/MyDrive/images"

4. Map each class in the dataset with the respective path to the data as a key:
 Value pair

In [ ]:
images_dict = {
    'bruises': os.path.join(dataset_path,'bruise'),
    'diabetic': os.path.join(dataset_path,'Diabetic'),

}

5. Map each  category of  image labels to numerical values for classification tasks

In [ ]:
images_labels ={
    'bruises': 0,
    'diabetic': 1

}

6. Perfom preprocessing on the data, shape resizing to 96x96

In [ ]:

X =[]
y=[]

for image_name , folder_path in images_dict.items():
  for image_file in os.listdir(folder_path):
    if image_file.endswith(".jpg")or image_file.endswith(".png"):
      image_path = os.path.join(folder_path, image_file)
      img= cv2.imread(image_path)
      resized_image= cv2.resize(img, (96, 96))
      X.append(resized_image)
      y.append(images_labels[image_name])

7. Convert to numpy array

In [ ]:

x= np.array(X)
y = np.array(y)

8. Create the train, test data samples  from the dataset

In [ ]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y ,test_size = 0.20,random_state=0)

9. Normalize the images

In [ ]:

x_train_scaled = X_train/255
x_test_scaled = X_test/255

9. Import MobilenetV2 for Transferlearning

In [ ]:

import tensorflow as tf

base_model = tf.keras.applications.MobileNetV2(input_shape= (96,96,3),
                                               include_top=False,
                                               alpha=0.35,
                                               weights='imagenet')

2019640/2019640 [==============================] - 1s 0us/step


10. Freeze the weights so they dont get updated during training

In [ ]:

for layer in base_model.layers:
  layer.trainable = False

11. Add a top layer with the number of classes and activation function

In [ ]:

from keras.layers import GlobalAveragePooling2D,Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Dropout

x= base_model.output
x= GlobalAveragePooling2D()(x)
x= Dropout(0.2)(x)
predictions = Dense(2, activation = 'softmax')(x)


In [ ]:
model = Model(inputs = base_model.input, outputs = predictions)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 Conv1 (Conv2D)                 (None, 48, 48, 16)   432         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 48, 48, 16)   64          ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 48, 48, 16)   0           ['bn_Conv1[0][0]']               
                                                                                              

12. Compile and then fit the model, stating the number of epochs and the validation split.

In [ ]:

learning_rate = 0.001
loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(
    optimizer = Adam(learning_rate = learning_rate),
    loss = loss,
    metrics = ['acc']
)

model.fit(x_train_scaled, y_train, epochs=10, validation_split = 0.2)

Epoch 1/10
16/16 [==============================] - 12s 149ms/step - loss: 0.5044 - acc: 0.7535 - val_loss: 0.2066 - val_acc: 0.9213
Epoch 2/10
16/16 [==============================] - 0s 14ms/step - loss: 0.2075 - acc: 0.9231 - val_loss: 0.1205 - val_acc: 0.9764
Epoch 3/10
16/16 [==============================] - 0s 13ms/step - loss: 0.1310 - acc: 0.9448 - val_loss: 0.0838 - val_acc: 0.9764
Epoch 4/10
16/16 [==============================] - 0s 11ms/step - loss: 0.0919 - acc: 0.9783 - val_loss: 0.0706 - val_acc: 0.9843
Epoch 5/10
16/16 [==============================] - 0s 12ms/step - loss: 0.0668 - acc: 0.9803 - val_loss: 0.0612 - val_acc: 0.9764
Epoch 6/10
16/16 [==============================] - 0s 12ms/step - loss: 0.0565 - acc: 0.9921 - val_loss: 0.0562 - val_acc: 0.9764
Epoch 7/10
16/16 [==============================] - 0s 12ms/step - loss: 0.0499 - acc: 0.9921 - val_loss: 0.0508 - val_acc: 0.9764
Epoch 8/10
16/16 [==============================] - 0s 15ms/step - loss: 0.0468 -

13. Evaluate the model, with unseen test data

In [ ]:
test_loss, test_acc = model.evaluate(x_test_scaled, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

5/5 [==============================] - 0s 11ms/step - loss: 0.0418 - acc: 1.0000
Test Loss: 0.04183685779571533
Test Accuracy: 1.0


14. Save the model in Keras Model format

In [ ]:

from tensorflow.keras.models import save_model
save_model(model, "model-quantized.h5")

15. Save the model as a tensorflow SavedModel format

In [ ]:


save_model_path= "/content/drive/MyDrive/tf-savedModel"
tf.saved_model.save(model,save_model_path)


16. Convert all the model weights from float32 to int8.
    then Convert the model to tesorflow lite format

In [ ]:


def representative_data_gen():
  """
    Generates representative data samples.

    This function takes a dataset, `X_train`, and yields preprocessed
    samples one at a time. Each sample is cast to the `tf.float32`
    data type. It ensures that only the first 200 samples are generated.

    Returns:
        A generator that yields preprocessed samples for TensorFlow.

    """
  for x in tf.data.Dataset.from_tensor_slices(X_train).batch(1).take(200):
    x = tf.cast(x, tf.float32)
    yield [x]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_model_quant = converter.convert()

# Save the quantized model to disk
with open('model_quantized.tflite', 'wb') as f:
    f.write(tflite_model_quant)

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:789: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
